### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Import Dataset using API token  from kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"thobelasixpence","key":"475e19ebcf14e6e7a3d7b6a98b3e06a0"}'}

In [3]:
# create a kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
# permission for json files to act
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

  0% 0.00/8.89M [00:00<?, ?B/s]
100% 8.89M/8.89M [00:00<00:00, 166MB/s]


In [6]:
!pip install zip_files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from zipfile import ZipFile
# open the zipped file
file_name="tmdb-movie-metadata.zip"
with ZipFile(file_name,'r') as zip:
      zip.extractall()
      print('Done')

Done


In [8]:
movies_df = pd.read_csv(r'/content/tmdb_5000_movies.csv')
credits_df = pd.read_csv(r'/content/tmdb_5000_credits.csv')

In [9]:
movies_df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [10]:
credits_df.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [11]:
movies_df.shape

(4803, 20)

In [12]:
credits_df.shape

(4803, 4)

### Merge Movies dataframe with credit dataframe

In [13]:
movies = movies_df.merge(credits_df,on='title')

In [14]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


We do not need the following  columns for this task:
- budget
- homepage
- id
- original_language
- original_title
- popularity
- production_comapny
- production_countries
- release-date(not sure)

In [15]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [16]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [17]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

### Format the text

#### format genre feature

In [18]:
import ast

In [19]:
# create a function that will get only the genre name of the movie
def get_genre_name(text):
  all_genre_names = []
  for i in ast.literal_eval(text):
    all_genre_names.append(i['name'])
  return all_genre_names

In [20]:
# drop the null values
movies.dropna(inplace=True)

In [21]:
# forma the genre feature
movies['genres'] = movies['genres'].apply(get_genre_name)
movies.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [22]:
def get_keywords_name(text):
  all_keywords_names = []
  for i in ast.literal_eval(text):
    all_keywords_names.append(i['name'])
  return all_keywords_names

In [23]:
# forma the keyword feature
movies['keywords'] = movies['keywords'].apply(get_keywords_name)
movies.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [24]:
# format the cast feature and get 3 cast members not more
def convert_cast_text(text):
  all_text = []
  counter = 0
  for i in ast.literal_eval(text):
    if counter<3:
      all_text.append(i['name'])
      counter+=1
    
  return all_text

In [25]:
movies['cast'] = movies['cast'].apply(convert_cast_text)
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [26]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [27]:
# get director of the movie
def get_director(text):
  directors=[]
  for i in ast.literal_eval(text):
    if i['job'] == 'Director':
      directors.append(i['name'])
  return directors

In [28]:
movies['crew'] = movies['crew'].apply(get_director)

In [29]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [30]:
# Eliminate white spaces between words
def eliminate_space(text):
  correct_format = []
  for i in text:
    correct_format.append(i.replace(" ",""))
  return correct_format

In [31]:
movies['genres'] = movies['genres'].apply(eliminate_space)
movies['keywords'] = movies['keywords'].apply(eliminate_space)
movies['cast'] = movies['cast'].apply(eliminate_space)
movies['crew'] = movies['crew'].apply(eliminate_space)

In [32]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [33]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [34]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [35]:
# combine all the features to create a single dataframe calles tags
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


In [36]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [37]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

In [38]:
new.head(2)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."


In [39]:
new['tags'] = new['tags'].apply(lambda x:x.lower())

In [40]:
new.head(2)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [41]:
import nltk

In [42]:
from nltk.stem import PorterStemmer # this library helps woth stemming process of finding the root of different words and place
# them under one column root of that word, to avoid having maultiple redundant words
ps = PorterStemmer()

In [43]:
def stem_words(text):
  words_list =[]
  for i in  text.split():
    words_list.append(ps.stem(i))

  return " ".join(words_list)

In [44]:
new['tags'] = new['tags'].apply(stem_words)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [46]:
# convert the  dataframe of tags to array
vector = cv.fit_transform(new['tags']).toarray()

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [48]:
# lets create a function that we will use for recommending movies
def recommend(movie_title):
  index = new[new['title']==movie_title].index[0]
  distances = sorted(list(enumerate(similarity[index])), reverse=True, key = lambda x: x[1])
  # only show the top 5
  for i in distances[1:6]:
    print(new.iloc[i[0]].title)

In [49]:
# test
recommend('Gandhi')

Gandhi, My Father
Guiana 1838
The Wind That Shakes the Barley
Mr. Turner
A Passage to India


In [50]:
# save the model and the movie list
import pickle

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
pickle.dump(new,open('/content/drive/MyDrive/Data/movie_list.pkl','wb'))
pickle.dump(similarity,open('/content/drive/MyDrive/Data/similarity.pkl','wb'))
